In [8]:
#---------------------------------------------------- IMPORTING PACKAGES------------------------------------------------
from tensorflow.keras.models import Sequential, load_model
# from tensorflow.keras.layers import Convolution2D, MaxPooling2D, Flatten, Dense, Dropout

from tensorflow.keras.preprocessing.image import load_img, img_to_array, array_to_img
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import numpy as np
import os
import cv2

from tensorflow.keras.preprocessing import image


import keras

from keras.layers import Dense, Conv2D , MaxPool2D , Flatten , Dropout , BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report,confusion_matrix

import seaborn as sns
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import utils
from tensorflow.keras import layers, models
from keras.layers import Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from keras.losses import CategoricalCrossentropy
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn import metrics
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import load_img, img_to_array, array_to_img
from tensorflow.keras.utils import to_categorical
from tkinter import *
import cv2
from PIL import ImageTk, Image

import mediapipe as mp

from tkinter import *
from PIL import ImageTk, Image
from gtts import gTTS 
from playsound import playsound
from pathlib import Path







In [9]:

# This is a Python class named Model that contains methods for building, saving, loading, and using a convolutional neural network (CNN) classifier for image classification. Here is a brief explanation of each method:

# __init__(self, Type) is the constructor that initializes the classifier attribute of the class with a specific type of CNN model specified by the Type argument.

# build_model(classifier) is a method that takes a CNN model as an input argument and adds layers to it. The layers consist of convolutional layers with ReLU activation, 
# batch normalization layers, max pooling layers, a flatten layer, and fully connected layers with ReLU activation and softmax activation for the output. The purpose of this method is to build the CNN model architecture.

# save_classifier(path, classifier) is a method that saves the trained CNN model to a file specified by the path argument.

# load_classifier(path) is a method that loads a trained CNN model from a file specified by the path argument.

# predict(classes, classifier, img) is a method that takes a list of class names (classes), a trained CNN model (classifier), 
# and an image (img) as input arguments. It resizes the image to 64x64 pixels, converts it to a NumPy array, normalizes the pixel 
# values to the range [0,1], performs a forward pass through the CNN model, and returns the predicted class label (as a string) and 
# the predicted class probabilities (as a NumPy array).

class Model:

  classifier = None
  def __init__(self, Type):
    self.classifier = Type
    
  
  def build_model(classifier):

    classifier.add(Conv2D(75 , (3,3) , strides = 1 , padding = 'same' , activation = 'relu' , input_shape = (64,64,1)))
    classifier.add(BatchNormalization())
    classifier.add(MaxPool2D((2,2) , strides = 2 , padding = 'same'))
    classifier.add(Conv2D(50 , (3,3) , strides = 1 , padding = 'same' , activation = 'relu'))
    classifier.add(Dropout(0.2))
    classifier.add(BatchNormalization())
    classifier.add(MaxPool2D((2,2) , strides = 2 , padding = 'same'))
    classifier.add(Conv2D(25 , (3,3) , strides = 1 , padding = 'same' , activation = 'relu'))
    classifier.add(BatchNormalization())
    classifier.add(MaxPool2D((2,2) , strides = 2 , padding = 'same'))
    classifier.add(Flatten())
    classifier.add(Dense(units = 512 , activation = 'relu'))
    classifier.add(Dropout(0.3))
    classifier.add(Dense(units = 29 , activation = 'softmax'))
   


    return classifier

  def save_classifier(path, classifier):
    classifier.save(path)

  def load_classifier(path):
    classifier = load_model(path)
    return classifier



  def predict(classes, classifier, img):
    img = cv2.resize(img, (64, 64))
    img = img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = img/255.0

    pred = classifier.predict(img)
    return classes[np.argmax(pred)], pred

In [10]:
# This is a Python class named DataGatherer that contains a method for loading and preprocessing images for image classification.
# Here is a brief explanation of the method:

# __init__(self, *args) is the constructor that initializes the dir attribute of the class with a directory path specified by the args
# argument. If no argument is passed, it initializes the dir attribute with an empty string.

# load_images(self) is a method that loads the images from the directory specified by the dir attribute, applies edge detection and resizing preprocessing functions on the images, and splits them into training and 
# testing datasets. It initializes empty lists for images and labels, loops over the folders in the dir directory and loads the images
# from each folder into the images list, and assigns a numerical label to each image based on its folder index. It then converts 
# the images and labels lists to NumPy arrays, normalizes the pixel values to the range [0,1], and one-hot encodes the labels. 
# Finally, it splits the images and labels arrays into training and testing datasets using a 90/10 split ratio and 
# returns the resulting x_train, x_test, y_train, and y_test arrays.

# edge_detection(self, image) is a helper method that takes an image as input and applies edge detection preprocessing on it. 
# The method converts the image to grayscale, applies Gaussian blur, adaptive thresholding, and Otsu thresholding to obtain a binary 
# image with edges detected, and returns the resulting image.

class DataGatherer:

  def __init__(self, *args):
    if len(args) > 0:
      self.dir = args[0]
    elif len(args) == 0:
      self.dir = ""


  #this function loads the images along with their labels and apply
  #pre-processing function on the images and finaly split them into train and
  #test dataset
  def load_images(self):
    images = []
    labels = []
    index = -1
    folders = sorted(os.listdir(self.dir))
    
    for folder in folders:
      index += 1
      
      print("Loading images from folder ", folder ," has started.")
      for image in os.listdir(self.dir + '/' + folder):

        img = cv2.imread(self.dir + '/' + folder + '/' + image,1)
        
        img = self.edge_detection(img)
        img = cv2.resize(img, (64, 64))
        img = img_to_array(img)

        images.append(img)
        labels.append(index)

    images = np.array(images)
    images = images.astype('float32')/255.0
    labels = to_categorical(labels)


    x_train, x_test, y_train, y_test = train_test_split(images, labels, test_size=0.1)

    return x_train, x_test, y_train, y_test

  def edge_detection(self, image):
    minValue = 70
    gray=cv2.cvtColor(image,cv2.COLOR_BGR2GRAY) 
    blur = cv2.GaussianBlur(gray,(5,5),2)
    th3 = cv2.adaptiveThreshold(blur,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY_INV,11,2)
    ret, res = cv2.threshold(th3, minValue, 255, cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
    return res

In [7]:
# FOR PRACTICAL DONT RUN THIS FILE



training_dir = '../dataset_3/asl_alphabet_train'
test_dir='../dataset_3/asl_test_1/asl-alphabet-test'

#loading the images from training directory
data_gatherer = DataGatherer(training_dir)


x_train, x_test, y_train, y_test = data_gatherer.load_images()

Loading images from folder  A  has started.
Loading images from folder  B  has started.


KeyboardInterrupt: 

In [ ]:
# batch_size, training_size, and test_size are initialized with some values.
# compute_steps_per_epoch is a lambda function that computes the number of steps per epoch based on the batch size and 
# the size of the training or test set.
# steps_per_epoch and val_steps are computed using the compute_steps_per_epoch lambda function and the training and test set sizes.
# An instance of the Model class is created, passing a Sequential object to the constructor.
# Then, build_model() method of the Model class is called to create the model, and the resulting model is assigned to the classifier
# variable.
# The compile() method is called on the classifier object to configure the learning process with the optimizer, loss function,
# and metrics.
# The fit() method is called on the classifier object to train the model on the training data. The steps_per_epoch, epochs, 
# validation_data, and validation_steps parameters are passed to the fit() method.
# The accuracy values during training and validation are stored in the history object returned by the fit() method.
# The accuracy graph is plotted using the matplotlib library.


batch_size = 64
training_size = x_train.shape[0]
test_size = x_test.shape[0]
#computing steps and validation steps per epoch according to training
#and testing size
compute_steps_per_epoch = lambda x: int(ceil(1. * x/batch_size))
steps_per_epoch = compute_steps_per_epoch(training_size)
val_steps = compute_steps_per_epoch(test_size)


#build the model
classifier = Model(Sequential()).classifier
classifier = Model.build_model(classifier)

classifier.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

#train the model
history = classifier.fit(
  x_train, y_train,
  steps_per_epoch=steps_per_epoch,
  epochs=10,
  validation_data=(x_test, y_test),
  validation_steps=val_steps)

#plot accuracy graph
plt.figure(figsize=(8,5))

plt.plot(history.history['accuracy'], label='train_accuracy',)
plt.plot(history.history['val_accuracy'], label='val_accuracy')
plt.xlabel('epoch')
plt.ylabel('accuracy')
plt.legend()
plt.title("classifier")

plt.show();

In [11]:
# This is a Python class called GUI that provides methods to create and customize a graphical user interface using the Tkinter library.

# __init__(self, title, size): The constructor method of the GUI class takes two arguments, a title string and a size string 
#     representing the dimensions of the window. It initializes the GUI by creating a new Tkinter object and 
#     setting the window's title and geometry.

# create_frame(self, width, height, anchor, relx, rely, background='white'): This method creates a new frame widget within the GUI window. It takes six arguments, including the width and height of the frame in pixels, the anchor position, and the relative x and y coordinates of the frame within the window. It also accepts an optional background parameter to set the background color of the frame.

# create_labels(self, label_num, labels, anchor, relx, rely, x_spacing=0, y_spacing=0, create_entrybox_per_label=False): 
#     This method creates one or more label widgets within the GUI window. It takes several arguments, 
#     including the number of labels to create, a list of label text strings, the anchor position, and the relative x and y coordinates 
#     of the first label. It also accepts optional parameters for x and y spacing between labels, and a flag to create a corresponding 
#     entry box widget for each label.

# create_buttons(self, button_num, text, anchor, relx, rely, command=None, x_spacing=0, y_spacing=0): This method creates one or
#     more button widgets within the GUI window. It takes several arguments, including the number of buttons to create, a list of button 
#     text strings, the anchor position, and the relative x and y coordinates of the first button. It also accepts optional parameters 
#     for a command to execute when the button is pressed, and x and y spacing between buttons.

# Overall, this class provides a simple way to create and customize a GUI window with labels, buttons, and frames using the Tkinter
# library.

class GUI:
    
    def __init__(self, title, size):
        self.root = Tk()
        self.root.title(title)
        self.root.geometry(size)

    def create_frame(self, width, height, anchor, relx, rely, background='white'):
        frame = Frame(self.root, bg=background, width=width, height=height)
        frame.place(anchor=anchor, relx=relx, rely=rely)
        return frame
        
    def create_labels(self, label_num, labels, anchor, relx, rely, x_spacing=0, y_spacing=0, create_entrybox_per_label=False):
        entry_labels = {}
        entry_boxes = {}
        relx = relx
        rely = rely

        longest_label_spacing = len(max(labels, key=len))/100.0
        
        for i in range(label_num):
            label = Label(self.root, text = labels[i]+": ",
                           font = ("TimesNewRoman", 15))
            label.place(anchor=anchor, relx=relx, rely=rely)
            
            entry_labels[labels[i]] = label
            if create_entrybox_per_label:
                entry_box = Text(self.root, font=("TimesNewRoman", 20), height=1, width=10)
                entry_box.place(anchor=anchor, relx=relx+longest_label_spacing+0.02, rely=rely)
                
                entry_boxes[labels[i]+'_entrybox'] = entry_box
            rely += y_spacing
            relx += x_spacing
        return entry_labels, entry_boxes

    def create_buttons(self, button_num, text, anchor, relx, rely, command=None, x_spacing=0, y_spacing=0):
        buttons = {}
        relx = relx
        rely = rely
        
        for i in range(button_num):
            btn = Button(self.root, command=command, text=text[i])
            btn.place(anchor=anchor, relx=relx, rely=rely)

            buttons[text[i]+' button'] = btn
            
            rely += y_spacing
            relx += x_spacing

        return buttons


In [12]:
# The code is an implementation of a sign language to text conversion application. 
# It uses a convolutional neural network to classify the hand gestures and then displays the text on the screen. 
# The user interface of the application is implemented using tkinter.

# The first few lines import the required modules and load the pre-trained CNN model. The code then defines the draw_region() function,
# which takes an image and a center point and returns a cropped version of the image centered around that point.

# The start_gui() function creates a GUI window and returns a reference to it along with a label to display the video feed. The exit_app() function is used to close the GUI window and release the video stream when the application is closed.

# The update_frame() function takes an image and a label and updates the label with the new image.

# The get_threshold() function extracts a float value from a tkinter Entry widget.

# The get_char() function takes a cropped image of a hand gesture, predicts the corresponding character using the CNN model, and returns the predicted character.

# The spell_app() function converts the text entered in the textbox into speech using the Google Text-to-Speech API and saves it as an audio file.

# The play() function plays the audio file.

# The AddCharToWord() function takes a word and a character and updates the word accordingly. 
# If the character is a space, the Auto_Correct() function is called to autocorrect the word. 
# If the character is "del," the last character of the word is deleted. If the character is not "nothing," 
# the character is added to the word.

# The frame_video_stream() function takes several arguments, including the names of the various tkinter widgets, 
# the current character, the previous character, the current word, and the current sentence. 
# It reads the video stream, crops the image around the center of the hand, and passes the cropped image to the CNN model 
# to predict the corresponding character. 
# It then updates the video feed with the predicted character and displays the current sentence on the screen.

mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands
cap = None

# classifier =load_model('newCnn_git_v3.h5',compile=False)
# classifier =load_model('newCnn_git_v4.h5')
# classifier =load_model('newCnn_git_v6_noAug.h5')
# classifier =load_model('newCnn_git_v7_Aug.h5')
classifier =load_model('newCnn_git_v10_black_white.h5')


def draw_region(image, center):
    cropped_image = cv2.rectangle(image, (center[0] - 130, center[1] - 130),
        (center[0] + 130, center[1] + 130), (0, 0, 255), 2)
    return cropped_image[center[1]-130:center[1]+130, center[0]-130:center[0]+130], cropped_image

def start_gui(title, size):
    gui = GUI(title, size)

    gui_frame = gui.create_frame(600, 600, 'ne', 1, 0, 'green')
    vid_label = Label(gui_frame)
    vid_label.grid()
    
    return gui, vid_label

def exit_app(gui, cap):
    gui.root.destroy()
    cap.release()


def update_frame(image, vid_label):
    image_fromarray = Image.fromarray(image)
    imgtk = ImageTk.PhotoImage(image=image_fromarray)
    
    vid_label.imgtk = imgtk
    vid_label.config(image=imgtk)

def get_threshold(label_entrybox):
    value = label_entrybox.get('1.0', END)
    try:
        return float(value)
    except:
        return 0.95


def get_char(gesture):
    classes = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L',
           'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V',
           'W', 'X', 'Y', 'Z', 'del', 'nothing', 'space']

    return Model.predict(classes, classifier, gesture)

def spell_app(sentbox):
    text=sentbox.get("1.0",'end-1c')
    language = 'en'
    # Passing the text and language to the engine,  
    # here we have marked slow=False. Which tells  
    # the module that the converted audio should  
    # have a high speed 
    myobj = gTTS(text=text, lang=language, slow=False) 

    # Saving the converted audio in a mp3 file named 
    path = Path('./Audio1.mp3')

    print(path.is_file())
    if(path.is_file()):
        os.remove("Audio1.mp3")
        myobj.save("Audio1.mp3")
       
        
    

    
    myobj.save("Audio1.mp3") 
def play():
    playsound('Audio1.mp3')
    


def AddCharToWord(word, curr_char):
    temp_word = word
    if curr_char == 'space':
        #print(Auto_Correct(temp_word))
        temp_word = ""
    elif curr_char == 'del':
        temp_word = temp_word[0:-1]
        print('character has been deleted')
    elif curr_char != 'nothing':
        temp_word += curr_char.lower()
        print('character has been added: ', curr_char.lower())
    else:
        pass

    return [temp_word, curr_char]


def frame_video_stream(names, curr_char, prev_char, word, sentence, *args):
    kwargs = dict(zip(names, args))
    
    threshold = get_threshold(kwargs['th_box'])
    curr_char = curr_char
    prev_char = prev_char
    
    success, frame = cap.read()
#     frame = cv2.flip(frame, 1)
    # Flip the image horizontally for a later selfie-view display, and convert
    # the BGR image to RGB.
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    update_frame(image, kwargs['vid_label'])

    image.flags.writeable = False
    results = kwargs['hands'].process(image)

    # Draw the hand annotations on the image.
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    image_height, image_width, _ = image.shape

    if results.multi_hand_landmarks:
        
        for hand_landmarks in results.multi_hand_landmarks:
            x = [landmark.x for landmark in hand_landmarks.landmark]
            y = [landmark.y for landmark in hand_landmarks.landmark]

            
            center = np.array([np.mean(x) * image_width, np.mean(y) * image_height]).astype('int32')
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            cropped_img, full_img = draw_region(image, center)

            update_frame(full_img, kwargs['vid_label'])

            try:
                #print('from try')
                # gray = cv2.cvtColor(cropped_img, cv2.COLOR_BGR2GRAY)
                gray = DataGatherer().edge_detection(cropped_img)

                curr_char, pred = get_char(gray)
                char = cv2.putText(full_img, curr_char, (center[0]-135, center[1]-135), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
                char_prob = cv2.putText(full_img, '{0:.2f}'.format(np.max(pred)), (center[0]+60, center[1]-135), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)

                update_frame(full_img, kwargs['vid_label'])

                kwargs['cc_box'].delete('1.0', 'end')
                kwargs['cc_box'].insert('end', curr_char)
                #print(curr_char)
                #compare the current char with the previous one and if matched, then won't add the current char
                #because the model catches the chars realy quick and if the below if statement removed,
                #the current char will be added endlessly to the word

                #also we use the threshold to prevent the meaningless characters to be added to the word
                #as the program catches the motion of the user's hand when the user changes the gesture(the motion between the gestures)
                #and the program thinks
                #it's a gesture and tries to match it with some letter but with low probability
                if (curr_char != prev_char) and (np.max(pred)> threshold):
                    #the below print statement is related to the formatter
                    #print(pred)
                    
                    temp = AddCharToWord(word, curr_char)
                    kwargs['ow_box'].insert('end', curr_char)
                    
                    if (temp[0] == "") and (temp[1] != "del"):
                        sentence+= Auto_Correct(word) + " "
                        kwargs['sent_box'].insert('end', Auto_Correct(word) + " ")
                        kwargs['ow_box'].delete('1.0', 'end')
                        kwargs['cw_box'].delete('1.0', 'end')
                        kwargs['cw_box'].insert('end', Auto_Correct(word))
                    word = temp[0]

                    prev_char = curr_char
            except:
                pass
    
    kwargs['vid_label'].after(1, frame_video_stream, names, curr_char, prev_char, word, sentence, *args)
    
    
    
def pipe_cam(gui, vid_label):
    
    curr_char = None
    prev_char = None
    word = ""
    sentence = ""

    
    
    
    #the predicted character won't be added to the word unless it's
    #probability is higher than the threshold
    #in places with good brightness and good camera the threshold can be a high value
    #otherwise it should be a low value and the reason for that is in places that meet
    #the above requirements, the model predict the letters with high probability to be
    #the correct letter the user ment to add
    threshold = float(0.95)

    

    #this formatter is to print the probability of the letters in readable
    #format just for the programmers if they want to see what are the probabilities looking like
    float_formatter = "{:.5f}".format
    np.set_printoptions(formatter={'float_kind':float_formatter})
    
    global cap
    cap = cv2.VideoCapture(0)

    labels_num = 5
    labels = ['threshold', 'current char', 'original word', 'corrected word', 'sentence']

    Labels, entryboxes = gui.create_labels(labels_num, labels, 'nw', 0, 0, y_spacing=0.06, create_entrybox_per_label=1)

    entryboxes['original word_entrybox'].config(width=18)
    entryboxes['corrected word_entrybox'].config(width=18)
    entryboxes['sentence_entrybox'].config(width=18, height=8)
    
    
    entryboxes['threshold_entrybox'].insert('end', threshold)
    th_entrybox = entryboxes['threshold_entrybox']


    cc_entrybox = entryboxes['current char_entrybox']


    ow_entrybox = entryboxes['original word_entrybox']


    cw_entrybox = entryboxes['corrected word_entrybox']


    sent_entrybox = entryboxes['sentence_entrybox']
    
    
    spell_program_btn = gui.create_buttons(1, ['CREATE SPEECH'], 'center', 0.2, 0.7, command=lambda: spell_app(entryboxes['sentence_entrybox']))
    play_program_btn = gui.create_buttons(1, ['PLAY'], 'center', 0.4, 0.8, command=lambda: play())

    
    Exit_program_btn = gui.create_buttons(1, ['Exit'], 'center', 0.5, 0.9, command=lambda: exit_app(gui, cap))

    names = ['vid_label', 'hands', 'th_box', 'cc_box', 'ow_box', 'cw_box', 'sent_box']
    with mp_hands.Hands(
            min_detection_confidence=0.4,
            min_tracking_confidence=0.5,
            max_num_hands=1) as hands:
        
            frame_video_stream(names, curr_char, prev_char, word, sentence, vid_label,
                               hands,  th_entrybox, cc_entrybox, ow_entrybox, cw_entrybox, sent_entrybox)
            gui.root.mainloop()


title = "Sign Language Recognition GUI"
size = "1100x1100"

gui, vid_label = start_gui(title, size)

pipe_cam(gui, vid_label)

1/1 [==============================] - 0s 36ms/step
character has been added:  t
1/1 [==============================] - 0s 50ms/step
character has been added:  y
1/1 [==============================] - 0s 46ms/step
character has been added:  t
1/1 [==============================] - 0s 45ms/step
character has been added:  f
1/1 [==============================] - 0s 49ms/step
character has been added:  y
1/1 [==============================] - 0s 45ms/step
character has been added:  a
1/1 [==============================] - 0s 40ms/step
